In [24]:
import obi_auth
from entitysdk import Client
from obi_auth import get_token
from obi_notebook import get_projects

# initialize token and project environment

In [25]:
token = get_token(environment="staging", auth_mode="daf")
project_context = get_projects.get_projects(token, env="staging")

Dropdown(description='Select:', options=(('Understanding SSCx', {'id': 'f373e771-3a2f-4f45-ab59-0955efd7b1f4',…

# get Client object.

In [26]:
client = Client(environment="staging", project_context=project_context, token_manager=token)

# create an EMDenseReconstructionDataset

In [29]:
from entitysdk.models import EMDenseReconstructionDataset,BrainRegion,Subject

brain_region = client.search_entity(entity_type=BrainRegion, query={"annotation_value": 68}).one()
subject = client.search_entity(entity_type=Subject).first()

em_dense_dataset = EMDenseReconstructionDataset(
      brain_region= brain_region,
      subject=subject,
      **{
     'name': 'this is an example',
     'description': 'this is a test dataset',
     'fixation': None,
     'staining_type': None,
     'slicing_thickness': 40.0,
     'tissue_shrinkage': None,
     'microscope_type': None,
     'detector': None,
     'slicing_direction': None,
     'landmarks': None,
     'voltage': None,
     'current': None,
     'dose': None,
     'temperature': None,
     'volume_resolution_x_nm': 4.0,
     'volume_resolution_y_nm': 4.0,
     'volume_resolution_z_nm': 40.0,
     'release_url': 'http://microns-explorer.org',
     'cave_client_url': 'https://global.daf-apis.com',
     'cave_datastack': 'minnie65_public',
     'precomputed_mesh_url': 'precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300/',
     'cell_identifying_property': 'pt_root_id'})
em_dense_reconstruction = client.register_entity(entity=em_dense_dataset, project_context=project_context)

In [30]:
em_dense_reconstruction = client.search_entity(entity_type=EMDenseReconstructionDataset, query={'name':'this is an example'} ).one()

# create an EMCellMesh

In [31]:
from entitysdk.models import EMCellMesh, EMCellMeshGenerationMethod, EMCellMeshType
em_mesh = EMCellMesh(
    generation_method=EMCellMeshGenerationMethod.marching_cubes,
    mesh_type=EMCellMeshType.dynamic,
    brain_region=brain_region,
    subject=subject,
    em_dense_reconstruction_dataset=em_dense_reconstruction,
    **{"release_version": 1512,
    "dense_reconstruction_cell_id": 12,
    "level_of_detail": 1,
    })
em_mesh_registered = client.register_entity(entity=em_mesh, project_context=project_context)

In [32]:
em_mesh_registered = client.search_entity(entity_type=EMCellMesh, query={'release_version':1512} ).one()

# uploading an .obj file

In [33]:
asset_obj = client.upload_file(
        entity_id=em_mesh_registered.id,
        entity_type=EMCellMesh,
        file_path="90.obj",
        file_content_type="application/obj",
        asset_label="cell_surface_mesh",
        project_context=project_context,
    )